<a href="https://colab.research.google.com/github/sbarreto10/data-science-2022/blob/main/IMDB%20RDD%20(TP2)/75_06_TP_2_MOVIES_RDD_SPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u342-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import numpy as np
from statistics import mean

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
rddMovies = SQLContext(sc).read.parquet("/content/drive/MyDrive/parquets tp2/movies.parquet").rdd
rddCrew = SQLContext(sc).read.parquet("/content/drive/MyDrive/parquets tp2/crew.parquet").rdd
rddLinks = SQLContext(sc).read.parquet("/content/drive/MyDrive/parquets tp2/links.parquet").rdd
rddCast = SQLContext(sc).read.parquet("/content/drive/MyDrive/parquets tp2/cast.parquet").rdd
rddRatings = SQLContext(sc).read.parquet("/content/drive/MyDrive/parquets tp2/ratings.parquet").rdd
rddCompanies = SQLContext(sc).read.parquet("/content/drive/MyDrive/parquets tp2/companies.parquet").rdd
rddKeywords = SQLContext(sc).read.parquet("/content/drive/MyDrive/parquets tp2/keywords.parquet").rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


*********************


CONSULTA S5


*********************







In [ ]:
rddCrewSFX = rddCrew.filter(lambda x: x.job == "Special Effects").map(lambda x: x.id)

In [ ]:
rddCrewSFXCount = rddCrewSFX.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
targetMovieId = rddCrewSFXCount.reduce(lambda x,y: x if x[1] > y[1] else y)[0]

In [ ]:
def ajustar_imdb_id(x):
    cant_ceros = 7 - len(x)
    imdb_id = "tt"
    for i in range(cant_ceros):
        imdb_id = imdb_id + "0"
    return imdb_id + x

In [ ]:
targetMovieImdbId = rddLinks.filter(lambda x: x.tmdbId == targetMovieId).first().imdbId
targetMovieImdbId = ajustar_imdb_id(str(targetMovieImdbId))

In [ ]:
targetMovie = rddMovies.filter(lambda x: x.imdb_id == targetMovieImdbId ).first().title

In [ ]:
print("La película con más trabajos en SFX es '" + targetMovie + "'")

La película con más trabajos en SFX es 'Jurassic World'


*********************


CONSULTA S28


*********************

In [ ]:
rddBradPittMovies = rddCast.filter(lambda x: x.name == "Brad Pitt").map(lambda x: x.id).distinct()

In [ ]:
rddLinksKv = rddLinks.map(lambda x: (x.tmdbId, x.movieId))
rddBradPittMoviesKv = rddBradPittMovies.map(lambda x: (x, 1))

In [ ]:
rddBradPittMoviesLinked = rddLinksKv.join(rddBradPittMoviesKv).map(lambda x: (x[1][0],x[0]))

In [ ]:
rddRatingsKv = rddRatings.map(lambda x: (x.movieId, (x.userId, x.rating)))
rddBradPittMovieRatings = rddRatingsKv.join(rddBradPittMoviesLinked)

In [ ]:
rddUserRatings = rddBradPittMovieRatings.map(lambda x: x[1][0])
rddUserRatingsMeans = rddUserRatings.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0],mean(x[1])))

In [ ]:
bestFanOfBradPitt = rddUserRatingsMeans.reduce(lambda x,y: x if x[1] > y[1] else y)[0]
print("El mayor fan de Brad Pitt es el usuario '" + str(bestFanOfBradPitt) + "'")

El mayor fan de Brad Pitt es el usuario '236204'


*********************


CONSULTA S31


*********************

In [ ]:
def ROI(r, b):
    try:
        return (float(r)-float(b))/float(b)
    except:
        return None

In [ ]:
rddROIMeans = rddMovies.map(lambda x: (x.production_companies, ROI(x.revenue,x.budget)))\
                    .filter(lambda x: x[0]!=None and x[1]!=None)\
                    .flatMap(lambda x: [(c, [x[1]]) for c in x[0].split(",")])\
                    .reduceByKey(lambda x,y: x+y).map(lambda x: (x[0], mean(x[1])))
majorROI = rddROIMeans.takeOrdered(3, lambda x: -x[1])
majorROIids = [int(c[0]) for c in majorROI]
majorROIcompanies = rddCompanies.filter(lambda x: any([ (x.id == i) for i in majorROIids ])).map(lambda x: x[1]).collect()

print("Las 3 compañías con mayor ROI promedio son:")
for c in majorROIcompanies:
    print(c)

Las 3 compañías con mayor ROI promedio son:
Amercent Films
American Entertainment Partners L.P.
Film It Suda


*********************


CONSULTA S27


*********************

In [ ]:
rddKeywordsKV = rddKeywords.filter(lambda x: x.keywords != None and x.id != None)\
                            .flatMap(lambda x: [(k, {x.id}) for k in x.keywords.split(",")])

In [ ]:
rddMoviesByKeyword = rddKeywordsKV.reduceByKey(lambda x,y: x.union(y))\
                                    .sample(False,0.2)\
                                        .map(lambda x: (x[0],(x[1],1)))
rddMBKCached = rddMoviesByKeyword.cache()
del [rddKeywordsKV, rddMoviesByKeyword]

In [ ]:
def dist(v, w):
    return 1 - len(v.intersection(w))/np.sqrt(len(v)*len(w))

def nearest(d, vx, vy):
    dx = dist(d[0], vx[1][0])
    dy = dist(d[0], vy[1][0])
    if dy%1==0: return (vx[0],(vx[1][0],dx))
    if dx%1==0: return (vy[0],(vy[1][0],dy))
    return (vx[0],(vx[1][0],dx)) if dx<dy else (vy[0],(vy[1][0],dy))

def farest(d, vx, vy):
    dx = dist(d[0], vx[1][0])
    dy = dist(d[0], vy[1][0])
    if dy%1==0: return (vx[0],(vx[1][0],dx))
    if dx%1==0: return (vy[0],(vy[1][0],dy))
    return (vx[0],(vx[1][0],dx)) if dx>dy else (vy[0],(vy[1][0],dy))

def rdd_nearest(dup, vRdd):
    n = vRdd.reduce(lambda x,y: nearest(dup, x, y))
    return (n[0], n[1][1])

def rdd_farest(dup, vRdd):
    f = vRdd.reduce(lambda x,y: farest(dup, x, y))
    return (f[0], f[1][1])

In [ ]:
# NO PUEDO HACER sets.map(lambda x: (x[0],rdd_nearest(x[1], sets),rdd_farest(x[1], sets)))
#   Entonces creo un diccionario previo al mapeado

In [ ]:
MBKList = rddMBKCached.collect()

In [ ]:
nearestDict = { k: rdd_nearest(dup, rddMBKCached) for (k,dup) in MBKList }

In [ ]:
farestDict = { k: rdd_farest(dup, rddMBKCached) for (k,dup) in MBKList }

In [ ]:
del [MBKList]
rddNeighbors = rddMBKCached.map(lambda x: (x[0],(nearestDict[x[0]], farestDict[x[0]])))\
                        .filter(lambda x: x[1][0][1]%1!=0 and x[1][1][1]%1!=0)
del [rddMBKCached]

In [ ]:
nearestList = rddNeighbors.takeOrdered(10, key=lambda x: -x[1][0][1])
farestList = rddNeighbors.takeOrdered(10, key=lambda x: -x[1][1][1])

In [ ]:
print("Los 10 pares de keywords más cercanas entre si:")
for x in nearestList:
    print("'" + x[0] + "' con '" + x[1][1][0] + "'" )
print("Los 10 pares de keywords más lejanas entre si:")
for x in farestList:
    print("'" + x[0] + "' con '" + x[1][1][0] + "'" )

Los 10 pares de keywords más cercanas entre si:
'counterfeiting' con 'woman director'
'photojournalist' con 'woman director'
'marital rape' con 'woman director'
'media hype' con 'woman director'
'partition' con 'woman director'
'nanjing' con 'woman director'
'oceanography' con 'independent film'
' stocks and bonds ' con 'independent film'
'homesick' con 'woman director'
'quarter' con 'woman director'
Los 10 pares de keywords más lejanas entre si:
'woman director' con 'killer'
'killer' con 'woman director'
'found footage' con 'woman director'
'haunted house' con 'woman director'
'chicago' con 'woman director'
'cult' con 'woman director'
'parody' con 'woman director'
'vigilante' con 'woman director'
'dragon' con 'woman director'
'treasure' con 'woman director'


*********************


CONSULTA S35


*********************

In [ ]:
def min(x,y):
    return x if x<y else y

def max(x,y):
    return x if x>y else y



In [ ]:
rddUserTimeCount = rddRatings.map(lambda x: (x.userId, ((x.timestamp, x.timestamp), 1))).sample(False,0.01)\
                            .reduceByKey(lambda x,y: ((min(x[0][0], y[0][0]), max(x[0][1], y[0][1])), x[1]+y[1]))\
                            .map(lambda x: (x[0], ((x[1][0][1] - x[1][0][0]), x[1][1])))
rddCached = rddUserTimeCount.cache()

oldestUser = rddCached.reduce(lambda x,y: x if x[1][0] > y[1][0] else y)[0]
(age, ageSquare, amount, amountSquare, amountAge) = rddCached\
                                    .map(lambda x: (x[1][0], x[1][0]*x[1][0], x[1][1], x[1][1]*x[1][1], x[1][0]*x[1][1]))\
                                .reduce(lambda x,y: (x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4]))


In [ ]:
n = rddCached.count()
SSXX = ageSquare - (1/n)*age*age
SSYY = amountSquare - (1/n)*amount*amount
SSXY = amountAge - (1/n)*amount*age
linearCorr = SSXY / np.sqrt(SSXX*SSYY)

In [ ]:
print("El id del usuario con más antigüedad es el " + str(oldestUser))

El id del usuario con más antigüedad es el 251197


In [ ]:
print("La correlación entre la antigüedad y la cantidad de ratings hechas es " + str(linearCorr))

La correlación entre la antigüedad y la cantidad de ratings hechas es 0.5171786758383117
